<a href="https://colab.research.google.com/github/RishnaChandran/kafkastream_test_pyspark/blob/main/kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install kafka-python

In [16]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
!curl -sSOL https://downloads.apache.org/kafka/2.8.1/kafka_2.12-2.8.1.tgz
!tar -xzf kafka_2.12-2.8.1.tgz

In [18]:
!./kafka_2.12-2.8.1/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-2.8.1/config/zookeeper.properties
!./kafka_2.12-2.8.1/bin/kafka-server-start.sh -daemon ./kafka_2.12-2.8.1/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [19]:
!ps -ef | grep kafka

root         528       1  0 06:20 ?        00:00:03 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-2.8.1/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-2.8.1/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-2.8.1/bin/../config/log4j.properties -cp /content/kafka_2.12-2.8.1/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-2.8.1/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-2.8.1/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-2.8.1/bin/../libs/audience-annotations-0.5.0.jar:/content/kafka_2.12-2.8.1/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.12-2.8.1/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.12

In [20]:
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic wordcount_test
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic wordcount_test_2

Error while executing topic command : Topic 'wordcount_test' already exists.
[2022-02-07 06:32:24,848] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'wordcount_test' already exists.
 (kafka.admin.TopicCommand$)
Error while executing topic command : Topic 'wordcount_test_2' already exists.
[2022-02-07 06:32:28,270] ERROR org.apache.kafka.common.errors.TopicExistsException: Topic 'wordcount_test_2' already exists.
 (kafka.admin.TopicCommand$)


In [21]:
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic wordcount_test
!./kafka_2.12-2.8.1/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic wordcount_test_2

Topic: wordcount_test	TopicId: _sdjJQpZQxOfABM6F0gD3w	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: wordcount_test	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
Topic: wordcount_test_2	TopicId: pZ9aULfmSiej8ihfjVYskg	PartitionCount: 2	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: wordcount_test_2	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: wordcount_test_2	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [23]:
!wget -O cm03FEB2022bhav.csv https://github.com/RishnaChandran/kafkastream_test_pyspark/blob/main/cm03FEB2022bhav.csv?raw=true

--2022-02-07 06:32:56--  https://github.com/RishnaChandran/kafkastream_test_pyspark/blob/main/cm03FEB2022bhav.csv?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/RishnaChandran/kafkastream_test_pyspark/raw/main/cm03FEB2022bhav.csv [following]
--2022-02-07 06:32:56--  https://github.com/RishnaChandran/kafkastream_test_pyspark/raw/main/cm03FEB2022bhav.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/RishnaChandran/kafkastream_test_pyspark/main/cm03FEB2022bhav.csv [following]
--2022-02-07 06:32:56--  https://raw.githubusercontent.com/RishnaChandran/kafkastream_test_pyspark/main/cm03FEB2022bhav.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.gi

In [26]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!pip install findspark

--2022-02-07 06:36:43--  https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.95.219, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300971569 (287M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop3.2.tgz’

spark-3.2.1-bin-had 100%[===================>] 287.03M  23.3MB/s    in 13s     

2022-02-07 06:36:56 (21.8 MB/s) - ‘spark-3.2.1-bin-hadoop3.2.tgz’ saved [300971569/300971569]

spark-3.2.1-bin-hadoop3.2/
spark-3.2.1-bin-hadoop3.2/LICENSE
spark-3.2.1-bin-hadoop3.2/NOTICE
spark-3.2.1-bin-hadoop3.2/R/
spark-3.2.1-bin-hadoop3.2/R/lib/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/DESCRIPTION
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/INDEX
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Meta/
spark-3.2.1-bin-hadoop3.2/R/lib/SparkR/Me

In [27]:
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar"

--2022-02-07 06:39:05--  https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-8-assembly_2.11/2.4.8/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12002039 (11M) [application/java-archive]
Saving to: ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’

spark-streaming-kaf 100%[===================>]  11.45M  --.-KB/s    in 0.1s    

2022-02-07 06:39:05 (115 MB/s) - ‘spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar’ saved [12002039/12002039]



In [30]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/spark-streaming-kafka-0-8-assembly_2.11-2.4.8.jar pyspark-shell'

In [31]:
import findspark
findspark.init()

In [34]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 48.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9843e1e8d654b8a24e3cf886d897c4825b94d9ae115dc9cf0849b01f097ae9f5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [57]:
import pyspark
import configparser
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Kafka streaming").getOrCreate()

In [42]:
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
nse_df = spark.read.format("csv") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load("cm03FEB2022bhav.csv")

In [43]:
display(nse_df)

DataFrame[SYMBOL: string, SERIES: string, OPEN: double, HIGH: double, LOW: double, CLOSE: double, LAST: double, PREVCLOSE: double, TOTTRDQTY: int, TOTTRDVAL: double, TIMESTAMP: string, TOTALTRADES: int, ISIN: string, _c13: string]

In [44]:
nse_df.show()

+----------+------+-------+-------+-------+--------+-------+---------+---------+-------------+-----------+-----------+------------+----+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|_c13|
+----------+------+-------+-------+-------+--------+-------+---------+---------+-------------+-----------+-----------+------------+----+
| 20MICRONS|    EQ|  91.65|   93.7|   89.5|    90.6|   91.0|     90.9|   260677| 2.37762701E7|03-FEB-2022|       4001|INE144J01027|null|
|21STCENMGM|    EQ|  42.05|  42.05|  42.05|   42.05|  42.05|     42.9|     3341|    140489.05|03-FEB-2022|         47|INE253B01015|null|
| 3IINFOLTD|    EQ|  77.35|   78.4|   76.6|    76.9|   76.9|    77.35|   374402|  2.8911018E7|03-FEB-2022|       4000|INE748C01038|null|
|   3MINDIA|    EQ|25221.5|25221.5|24520.8|24868.65|24580.0| 24982.75|     3570|8.881929265E7|03-FEB-2022|       2078|INE470A01017|null|
|    3PLAND|    BE|   19.0|   19.6|   19.

In [47]:
df = nse_df.drop("_c13").show()

+----------+------+-------+-------+-------+--------+-------+---------+---------+-------------+-----------+-----------+------------+
|    SYMBOL|SERIES|   OPEN|   HIGH|    LOW|   CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+-------+-------+-------+--------+-------+---------+---------+-------------+-----------+-----------+------------+
| 20MICRONS|    EQ|  91.65|   93.7|   89.5|    90.6|   91.0|     90.9|   260677| 2.37762701E7|03-FEB-2022|       4001|INE144J01027|
|21STCENMGM|    EQ|  42.05|  42.05|  42.05|   42.05|  42.05|     42.9|     3341|    140489.05|03-FEB-2022|         47|INE253B01015|
| 3IINFOLTD|    EQ|  77.35|   78.4|   76.6|    76.9|   76.9|    77.35|   374402|  2.8911018E7|03-FEB-2022|       4000|INE748C01038|
|   3MINDIA|    EQ|25221.5|25221.5|24520.8|24868.65|24580.0| 24982.75|     3570|8.881929265E7|03-FEB-2022|       2078|INE470A01017|
|    3PLAND|    BE|   19.0|   19.6|   19.0|    19.6|   19.6|     18.7|     4

In [48]:
c=0

header = {
    'Accept-Encoding': 'gzip, deflate, sdch, br',
    'Accept-Language': 'fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4',
    'Host': 'www1.nseindia.com',
    'Referer': 'https://www1.nseindia.com/',
    'User-Agent': 'Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/53.0.2785.143 Chrome/53.0.2785.143 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'}

In [49]:
def calculateThoughput(timeDiff, msg_count, msg_size):
    log.info("Processed {0} messsages in {1:.2f} seconds".format(msg_count, timeDiff))
    log.info("{0:.2f} MB data processed".format(((msg_size) / (1024 * 1024))))
    log.info("{0:.2f} MB/s".format(((msg_size) / (1024 * 1024)) / timeDiff))
    log.info("{0:.2f} Msgs/s".format(msg_count / timeDiff))

In [50]:
def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)

In [51]:
#Create kafka Topic
def createKafkaTopic(topic_name,PARTITIONS,REPLICATION_FACTOR,KAFKA_BROKERS):
    admin_client = KafkaAdminClient(bootstrap_servers=KAFKA_BROKERS)
    topic_list = [NewTopic(name=topic_name, num_partitions=PARTITIONS, replication_factor=REPLICATION_FACTOR)]
    admin_client.create_topics(new_topics=topic_list, validate_only=False)


In [53]:
def data(df):
    data=df.to_json(orient='index')
    data=json.loads(data)
    i=0
    while i<=len(data)-1:
        yield data[str(i)]
        i+=1


In [54]:
#Scrapes data from website and writes to kafka Topic
def scrape(link,year,producer,day,month,TOPIC_NAME,partitionId):
    startTime=time.time()
    try:
        req = Request(link, headers=header)
        request = urlopen(req, timeout=5)
        status_code = request.getcode()
        if status_code==200:
            zip_file = zipfile.ZipFile(BytesIO(request.read()))
            names = zip_file.namelist()
            for name in names:
                df=pd.read_csv(zip_file.open(name))
                try:
                    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
                except:
                    pass
                msg_size=0
                msg_count=df.shape[0]
                for record in data(df):
                    producer.send(TOPIC_NAME, value=record, partition=partitionId).add_errback(on_send_error)
                    #.add_callback(on_send_success)
                    msg_size+=int(sys.getsizeof(str(record).encode("utf-8")))
                producer.flush()
                #metrics=producer.metrics()
                #print(metrics)
                endTime=time.time()
                timeDiff=endTime-startTime
                log.info(f"{year}-{month}-{day} data inserted to topic-{TOPIC_NAME} of partition-{partitionId}")
                calculateThoughput(timeDiff, msg_count, msg_size)
                #print(f"INSERTED {msg_count} records in {timeDiff:.2f} seconds")
                #print(f'inserted count-{c}')
    except:
        log.info(f"{year}-{month}-{day} is NSE holiday")

In [62]:
from google.colab import files
upload = files.upload()

Saving config.ini to config.ini


In [65]:
import configparser
def main():
    config = configparser()
    config.read('config.ini')
    DATA_FROM_YEAR=config.get('kafkaProducer','scrapeDataFromYear')
    KAFKA_BROKERS=config.get('kafkaProducer','kafka_bootstrap_servers').split(',')
    TOPIC_NAME=config.get('kafkaProducer','topic')
    PARTITIONS=config.getint('kafkaProducer','partitions')
    REPLICATION_FACTOR=config.getint('kafkaProducer','replications')
    lastdate=f"{DATA_FROM_YEAR}-01-01"
    now=datetime.now().strftime("%Y-%m-%d")
    start = datetime.strptime(str(lastdate), "%Y-%m-%d")
    end = datetime.strptime(now, "%Y-%m-%d")
    date_generated = [start + timedelta(days=x) for x in range(1, (end-start).days+1)]
    print("Producer starting")
    try:
        producer = KafkaProducer(bootstrap_servers=KAFKA_BROKERS,value_serializer=lambda x: dumps(x).encode('utf-8'))
        print("Producer started")
    except Exception as e:
        log.exception(e)
        #raise e
    try:
        topicCreation=createKafkaTopic(TOPIC_NAME, PARTITIONS, REPLICATION_FACTOR, KAFKA_BROKERS)
        print(f'{TOPIC_NAME} is created')
    except Exception as e:
        log.exception(e)
        raise e

    partitionYearsandId = []
    tempPartitionId = 0
    for date in date_generated:
        day=date.strftime("%d")
        month=(date.strftime("%B")[:3]).upper()
        year=date.strftime("%Y")
        if not any(year in x for x in partitionYearsandId):
            partitionYearsandId.append({year:tempPartitionId})
            tempPartitionId+=1
        partitionId=[x.get(year) for x in partitionYearsandId if x.get(year)!=None][0]
        fname="cm"+str(day)+str(month)+str(year)+"bhav.csv.zip"
        link="https://www1.nseindia.com/content/historical/EQUITIES/"+str(year)+"/"+month+"/"+fname
        scrape(link,year,producer,day,month,TOPIC_NAME,partitionId)

if __name__ == '__main__':
    main()

TypeError: ignored

In [64]:
if __name__ == '__main__':
    main()

TypeError: ignored